Task 4 : Bengali

Glove embedding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

Pre-processing

In [ ]:
!pip install emot --upgrade
!pip install emoji
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=1d33f5ae77ccade929936c4bf7cad9cba362f84add4a33f9eeae296f7a4fa769
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 896.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.metrics import precision_recall_fscore_support as score
from textblob import TextBlob
import contractions
import emot
import emoji
import demoji
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import nltk
nltk.download('omw-1.4')
pd.set_option("max_colwidth" ,120)
#wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
#stopword = nltk.corpus.stopwords.words('english')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/bn.txt', encoding = 'utf-8') as f:
    ben_stopwords = f.readlines()
    for i in range(len(ben_stopwords)):
        ben_stopwords[i] = re.sub('\n','', ben_stopwords[i])
stopwords = english_stopwords + ben_stopwords
#english_stemmer = SnowballStemmer("english")

In [ ]:
len(stopwords)

298

In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-13-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
punct='“!"#$%&\'।()*+,-./:;<=>?@[\\]^_`{|}~”'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '',text)#remove @ - A Z
    text=contractions.fix(text,slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    text= "".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)#this is for not splitting other languages but it is not removing emojis
    if len(text) == 0:
      text= '**********************'
    return text

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/train_BE_AH_HASOC2023.csv')
df2=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/test_BE_AH_HASOC2023.csv')

In [ ]:
df1.head()

,S. No.,text,task_1
0,1,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দিয়েছে তাহলে ব্রাত্য বসুর মুখে কোটা কেন? এটা কথার কথা? তাহলে কোটা নিয়ে...,NOT
1,2,"প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ, এমপি, এদের একটা নিজস্ব কোটা আছে, সেই কোটায় তারা যাকে খুশি চাকরি দিতে পারেন, এ...",NOT
2,3,"বিশ্ব জুড়ে বুদ্ধিজীবিদের সঙ্গে সম্পর্ক থাকে বিশ্ববিদ্যালয়,সেমিনার, পাবলিশিং হাউস ও মিডিয়ার। বাংলায় দেখছি গরু চোরাক...",HOF
3,4,প্রতি বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতান্ত্রিক আর আইনের শাসনের দেশ ভারতের সীমান্তরক্ষীরা। পৃথিবীর কোন সভ্য দে...,NOT
4,5,সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ করে দেয় সমাজকে ছি ছি ছি,HOF


In [ ]:
df2.head()

,S. No.,text
0,1,তাতে তোর কী? বেশি চুলকুনি নারে সালা ?
1,2,অবশ্য চটি চাটা হতে হবে
2,3,"KKR এর প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া কি বাধ্যতামূলক? শাহরুখ KKR এর সহ-মালিক হতে পারে, ক্রিকেটার নয়। ক্রিকেট এবং ক্..."
3,4,হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে। যার যা নাম ধরে ডাকলেই চিনতে পারে।
4,5,এটা নিজের মেয়ের নাম রাখবেন আর এই নাম ধরে ডাকবেন .....😁😅😆


In [ ]:
df1['clean_text'] = df1['text'].apply(lambda x:emo(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:remove_urls(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df2['clean_text'] = df2['text'].apply(lambda x:emo(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:remove_urls(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df1['clean_text'][0:10]

0    শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দিয়েছে তাহলে ব্রাত্য বসুর মুখে কোটা এটা কথার কথা তাহলে কোটা নিয়ে প্রাক্তন ...
1    প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ এমপি এদের একটা নিজস্ব কোটা কোটায় তারা যাকে খুশি চাকরি দিতে পারেন এবং চিরকাল এন...
2    বিশ্ব জুড়ে বুদ্ধিজীবিদের সম্পর্ক থাকে বিশ্ববিদ্যালয়সেমিনার পাবলিশিং হাউস মিডিয়ার বাংলায় দেখছি গরু চোরাকারবারিদের স...
3    বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতান্ত্রিক আইনের শাসনের দেশ ভারতের সীমান্তরক্ষীরা পৃথিবীর কোন সভ্য দেশের সীমান্...
4                                                                  সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ দেয় সমাজকে ছি ছি ছি
5    যারা প্রেরণার কথার মানে বুঝতে পারেনি তাদের মানে বোঝার জন্য ওর যেতে হবে ঠাণ্ডা মস্তিস্ক নিয়ে ভালো বোঝাতে পারবে আমার ...
6                                                                         কন্ডম দিদি grinning face sweat grinning face sweat
7                                                                                                         বড় বড় কথা কোথায়


In [ ]:
df2['clean_text'][0:10]

0                                                                                              তাতে তোর কী চুলকুনি নারে সালা
1                                                                                                               চটি চাটা হবে
2    kkr প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া বাধ্যতামূলক শাহরুখ kkr সহমালিক পারে ক্রিকেটার ক্রিকেট এবং ক্রিকেটারকে ভালবেসে আপ...
3                                               হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে যার নাম ধরে ডাকলেই চিনতে পারে
4            এটা নিজের মেয়ের নাম রাখবেন নাম ধরে ডাকবেন beaming face smiling eyes grinning face sweat grinning squinting face
5    পিগি চপস মানে বাংলায় ওইটাই বোঝায় face tears joy face tears joy face tears joy face tears joy face tears joy আমার ম...
6                                                                                                  মাছেলে দুজনকেই ভালো লাগছে
7                                                                         ইয়ে মাথায় ওঠেনি এগুলো সবই পরিকল্পনা বিদ্বেষ ছাড়ানো


In [ ]:
#split the dataset into train and test sets
X_train1, X_test1= train_test_split(df1.clean_text,test_size=0.25, random_state=42)

In [ ]:
X_train1.shape, X_test1.shape

((960,), (321,))

Loading Glove embeddings(bengali and english)

In [ ]:
embedding_index_ben = {}
benvocab = []
with open('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/bn_glove.39M.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        benvocab.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_ben[word] = coefs

In [ ]:
len(benvocab)

178153

In [ ]:
def get_vectors(sentence):
    sentvec = np.zeros(300)
    zeros = np.zeros(300)

    for word in sentence.split():
        if word in benvocab:
            temp = embedding_index_ben[word]
        else:
            temp = zeros
        sentvec += temp

    sentvec = sentvec / len(sentence.split())
    return sentvec

In [ ]:
#vectors
X_train1['vectors'] = df1['clean_text'].apply(lambda x:get_vectors(x))

In [ ]:
X_test1['vectors'] = df1['clean_text'].apply(lambda x:get_vectors(x))

In [ ]:
#converting to list.
train=X_train1['vectors'].to_list()
test=X_test1['vectors'].to_list()

In [ ]:
y_train1=df1['task_1'].to_list()
y_test1=df1['task_1'].to_list()

SVM model

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
svm_model = SVC(kernel='linear')

In [ ]:
svm_model.fit(train, y_train1)

SVC(kernel='linear')

In [ ]:
y_pred = svm_model.predict(test)

In [ ]:
accuracy = accuracy_score(y_test1, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test1, y_pred))

Test Accuracy: 77.3614

               precision    recall  f1-score   support

         HOF       0.77      0.62      0.69       515
         NOT       0.77      0.88      0.82       766

    accuracy                           0.77      1281
   macro avg       0.77      0.75      0.76      1281
weighted avg       0.77      0.77      0.77      1281



**Evaluation for test set**

In [ ]:
df1['vectors'] = df1['clean_text'].apply(lambda x:get_vectors(x))

In [ ]:
df2['vectors'] = df2['clean_text'].apply(lambda x:get_vectors(x))

In [ ]:
X_train=df1['vectors'].tolist()
X_test=df2['vectors'].tolist()
y_train=df1['task_1'].tolist()

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
svm_model = SVC(kernel='linear')

In [ ]:
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred1 = svm_model.predict(X_test)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred1, columns=['y_pred1'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run6.csv',index = None)